### HEROKUのポスグレの接続テスト

In [1]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()



#自動コミットモードにする

conn.autocommit = True
conn.autocommit

True

In [18]:
cursor.execute('''current_date ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
maptable_df['table_name'] = 'maptable'
maptable_df

SyntaxError: syntax error at or near "current_date"
LINE 1: current_date 
        ^


In [77]:
def get_area_sql_text(target_area_name):
    print(target_area_name)
    hokkaidoutouhoku_list = ['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県']
    kitakantou_list = ['茨城県', '栃木県', '群馬県']
    minamikantou_list = ['埼玉県', '千葉県', '東京都', '神奈川県']
    hokurikukoushinetsu_list = ['新潟県', '富山県', '石川県', '福井県', '長野県', '山梨県']
    toukai_list = ['愛知県', '岐阜県', '静岡県', '三重県']
    kansai_list = ['大阪府', '京都府', '兵庫県', '奈良県', '滋賀県', '和歌山県']
    chugokushikoku_list = ['鳥取県', '島根県', '岡山県', '広島県','徳島県', '香川県', '愛媛県', '高知県']
    kyushu_list = [ '山口県','福岡県', '佐賀県', '長崎県', '大分県', '熊本県', '宮崎県', '鹿児島県','沖縄県']
    prefecture_str_lists = ['hokkaidoutouhoku_list', 'kitakantou_list','minamikantou_list','hokurikukoushinetsu_list','toukai_list','kansai_list','chugokushikoku_list','kyushu_list'] 
    for prefecture_list_name in prefecture_str_lists:
        #print(prefecture_list_name)
        if target_area_name == prefecture_list_name.replace('_list',''):
            print('ok',target_area_name)
            target_area_name_list = eval(prefecture_list_name)
            #print(target_area_name_list)
            break
    area_sql_text = ''
    for prefecture_name in target_area_name_list:
        if len(area_sql_text) == 0:
            area_sql_text += f"都道府県 = '{prefecture_name}'"
        else:
            area_sql_text += f" OR 都道府県 = '{prefecture_name}'"
    #print(area_sql_text)
    return area_sql_text

area_sql_text = get_area_sql_text('kitakantou')

print(area_sql_text)

kitakantou
ok kitakantou
都道府県 = '茨城県' OR 都道府県 = '栃木県' OR 都道府県 = '群馬県'


In [58]:
minami_kantou_list:list[str] = ['埼玉県', '千葉県', '東京都', '神奈川県']



都道府県 = '埼玉県' OR 都道府県 = '千葉県' OR 都道府県 = '東京都' OR 都道府県 = '神奈川県'


In [55]:
#首都圏のイベントの媒体別の予約数を集計
media_name = 'スロパチステーション'
cursor.execute(f'''SELECT イベント日,都道府県,店舗名,取材名
               FROM schedule
               WHERE イベント日 >= current_date
                AND イベント日 < current_date + 7
                AND 媒体名 != 'ホールナビ'
                AND 媒体名 = '{media_name}'
                AND (都道府県 = '東京都' OR 都道府県 = '神奈川県' OR 都道府県 = '埼玉県' OR 都道府県 = '千葉県')
                ORDER BY イベント日,都道府県 desc;''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
extract_media_name_df

,イベント日,都道府県,店舗名,取材名
0,2023-08-10,神奈川県,マルハン川崎桜本店,かたまる応援地区
1,2023-08-10,神奈川県,マルハン茅ヶ崎店,ぱちまる応援地区
2,2023-08-10,神奈川県,マルハン厚木店,かたまる応援地区
3,2023-08-10,神奈川県,新！ガーデン戸塚,スロパチ広告
4,2023-08-10,神奈川県,マルハン平塚店,ぱちまる応援地区
5,2023-08-11,神奈川県,アビバ関内店,かたまる応援地区
6,2023-08-11,神奈川県,ピーアーク相模原,かたまる応援地区
7,2023-08-11,神奈川県,アビバANNEX&SQUARE,かたまる応援地区
8,2023-08-11,神奈川県,ピーアーク相模大野,かたまる応援地区
9,2023-08-11,神奈川県,マルハンメガシティ横浜町田,ぱちまる応援地区


In [68]:
cursor.execute('''SELECT dmm_name,hallnavi_name, prefecture, latitude
               FROM maptable
               WHERE prefecture = '神奈川県' 
               OR prefecture = '東京都'
                OR prefecture = '埼玉県'
                OR prefecture = '千葉県'
               ORDER BY prefecture''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
maptable_df['table_name'] = 'maptable'
maptable_df

,dmm_name,hallnavi_name,prefecture,latitude,table_name
0,ダイナム八街店,ダイナム八街店,千葉県,35.669998,maptable
1,ピーアーク南行徳NEO,ピーアーク南行徳NEO,千葉県,35.672798,maptable
2,アムディ東松戸,アムディ東松戸,千葉県,35.776299,maptable
3,パールショップともえ小見川店,パールショップともえ小見川店,千葉県,35.848598,maptable
4,パールショップともえ町田609,パールショップともえ成田969,千葉県,35.541698,maptable
...,...,...,...,...,...
569,大学院,大学院,神奈川県,35.521301,maptable
570,プラザ上大岡アンドミラージュ,プラザ上大岡アンドミラージュ,神奈川県,35.410301,maptable
571,相模大野UNO,相模大野UNO,神奈川県,35.532200,maptable
572,小田急相模原UNO,小田急相模原UNO,神奈川県,35.515499,maptable


In [67]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE 都道府県 = '神奈川県' 
               OR 都道府県 = '東京都'
                OR 都道府県 = '埼玉県'
                OR 都道府県 = '千葉県'
               ORDER BY 都道府県''')
cols = [col.name for col in cursor.description]
tenpo_master_df = pd.DataFrame(cursor.fetchall(),columns=cols)
tenpo_master_df['table_name'] = 'tenpo_master'
tenpo_master_df.rename(columns={'店舗名／法人名':'dmm_name','都道府県':'prefecture'},inplace=True)
tenpo_master_df

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name
0,エクス・アリーナ 松戸,千葉県,35.762413,139.93720,千葉県松戸市高塚新田326‐5,tenpo_master
1,プリンス,千葉県,34.973930,137.16154,愛知県岡崎市井田南町7-7,tenpo_master
2,イーグルアール富里店,千葉県,35.749073,140.30844,千葉県富里市七栄1001-10,tenpo_master
3,キクヤ千葉店,千葉県,35.587177,140.12975,千葉県千葉市中央区末広5-7-1,tenpo_master
4,ＳＫＩＰ 館山店,千葉県,35.009476,139.87060,千葉県館山市湊231,tenpo_master
...,...,...,...,...,...,...
1215,ＨＡＰＰＹ Ｇ・ＦＵＪＩＳＡＷＡ,神奈川県,35.340324,139.48819,神奈川県藤沢市藤沢556,tenpo_master
1216,パラッツォ藤沢南店,神奈川県,35.338270,139.48947,神奈川県藤沢市南藤沢5-13,tenpo_master
1217,パールショップともえ平塚６０６,神奈川県,35.328053,139.34818,神奈川県平塚市紅谷町8-16,tenpo_master
1218,トワーズ藤沢店,神奈川県,35.403885,139.42984,神奈川県藤沢市葛原1806-6,tenpo_master


In [70]:
concat_df = pd.concat([tenpo_master_df,maptable_df],axis=0)
concat_df 

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name,hallnavi_name,latitude
0,エクス・アリーナ 松戸,千葉県,35.762413,139.93720,千葉県松戸市高塚新田326‐5,tenpo_master,NaN,NaN
1,プリンス,千葉県,34.973930,137.16154,愛知県岡崎市井田南町7-7,tenpo_master,NaN,NaN
2,イーグルアール富里店,千葉県,35.749073,140.30844,千葉県富里市七栄1001-10,tenpo_master,NaN,NaN
3,キクヤ千葉店,千葉県,35.587177,140.12975,千葉県千葉市中央区末広5-7-1,tenpo_master,NaN,NaN
4,ＳＫＩＰ 館山店,千葉県,35.009476,139.87060,千葉県館山市湊231,tenpo_master,NaN,NaN
...,...,...,...,...,...,...,...,...
569,大学院,神奈川県,NaN,NaN,NaN,maptable,大学院,35.521301
570,プラザ上大岡アンドミラージュ,神奈川県,NaN,NaN,NaN,maptable,プラザ上大岡アンドミラージュ,35.410301
571,相模大野UNO,神奈川県,NaN,NaN,NaN,maptable,相模大野UNO,35.532200
572,小田急相模原UNO,神奈川県,NaN,NaN,NaN,maptable,小田急相模原UNO,35.515499


In [86]:
len(maptable_df.columns)

11

In [85]:
t = ''
for column_name in maptable_df.columns:
    t += f'{column_name} ,'
print(t)

parlar_map_id ,prefecture ,kintone_id ,shop_id ,hallnavi_name ,dmm_name ,anaslo_name ,latitude ,longitude ,adress ,created_at ,


In [71]:
dupulicated_concat_df = concat_df[concat_df.duplicated(['dmm_name'],keep=False)].sort_values(['dmm_name'],ascending=False)
dupulicated_concat_df

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name,hallnavi_name,latitude
187,ＳＡＰ野田,千葉県,35.941727,139.84975,千葉県野田市中野台815,tenpo_master,NaN,NaN
42,ＳＡＰ野田,千葉県,NaN,NaN,NaN,maptable,SAP野田,35.941700
41,ＳＡＰ野田,千葉県,NaN,NaN,NaN,maptable,SAP野田,35.941700
111,Ｍｙピノキオ浦和西店,埼玉県,NaN,NaN,NaN,maptable,Myピノキオ浦和西店,35.861099
246,Ｍｙピノキオ浦和西店,埼玉県,35.861140,139.62779,埼玉県さいたま市桜区南元宿2-8-15,tenpo_master,NaN,NaN
...,...,...,...,...,...,...,...,...
1130,PIA伊勢佐木町,神奈川県,35.444836,139.63373,神奈川県横浜市中区伊勢佐木町1-2-3ピアステーション1~4F,tenpo_master,NaN,NaN
455,PIA京急川崎,神奈川県,NaN,NaN,NaN,maptable,PIA京急川崎,35.532299
1031,PIA京急川崎,神奈川県,35.532320,139.69980,神奈川県川崎市川崎区駅前本町16-1,tenpo_master,NaN,NaN
472,JUMBO,神奈川県,NaN,NaN,NaN,maptable,JUMBO上大岡,35.410198


In [ ]:
a_list = list(tenpo_master_df['dmm_name'].unique())
b_list = list(dupulicated_concat_df[dupulicated_concat_df['table_name'] == 'maptable']['dmm_name'].unique())
serch_list = list(set(a_list) - set(b_list))
serch_list

In [66]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度
               FROM tenpo_master
               WHERE 経度 is NULL ''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度
0,アンダーツリー株式会社,大阪府,None,None
1,コシード株式会社,,None,None
2,株式会社千歳観光,,None,None
3,BASE株式会社,,None,None
4,ガイドワークス,,None,None
...,...,...,...,...
114,トキワホール,,None,None
115,パーラーデルダス,,None,None
116,Super D’station前橋大利根店,群馬県,None,None
117,yout小牧店,愛知県,None,None


In [222]:
dmm_tenpo_name.replace("'", "''")

"Super D''station錦糸町店"

In [ ]:
for i,row in serched_df.iterrows():
    print(row['longtitude'])
    longtitude = row['longtitude']
    latitude = row['latitude']
    dmm_tenpo_name = row['dmm_tenpo_name'].replace("'", "''")
    try:
        cursor.execute(f"""UPDATE tenpo_master
                    SET 経度 = {longtitude}, 緯度 = {latitude}
                WHERE  店舗名／法人名 = '{dmm_tenpo_name}' """)
        conn.commit()
    except Exception as e: 
        print("error", dmm_tenpo_name)
        break



In [194]:
cursor.execute(f'''UPDATE tenpo_master
                SET 経度 = {longtitude}, 緯度 = {latitude}
               WHERE  店舗名／法人名 = 'ＳＬＯＴ Ｇ８' ''')

In [7]:
cursor.execute(f'''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE  店舗名／法人名 = 'エムズファクトリー(千葉)' ''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度,住所(ホールデータ)
0,エムズファクトリー(千葉),千葉県,35.790844,139.93799,


In [ ]:
serched_df = pd.read_csv('csv/scrapinged_kanto_tenpo_name_table.csv')
serched_df

In [81]:

cursor.execute('''SELECT *
               FROM maptable
               WHERE prefecture = '神奈川県' AND latitude is NULL ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)





,parlar_map_id,prefecture,kintone_id,shop_id,hallnavi_name,dmm_name,anaslo_name,latitude,longitude,adress,created_at
0,262,神奈川県,P12734,13166,アビバ逗子駅前店,アビバ逗子駅前店,アビバ逗子駅前店,None,None,神奈川県逗子市逗子5-2-46,2023-03-18 05:36:49
1,1526,神奈川県,P04795,4795,アビバ湘南台店,アビバ湘南台店,アビバ湘南台店,None,None,神奈川県藤沢市湘南台1-2-1,2023-03-18 05:37:55


In [ ]:
cursor = conn.cursor()
cursor.execute('''SELECT COUNT(DISTINCT 店舗名)
               FROM parlar_data
               WHERE 都道府県 = '神奈川県' or '埼玉県' ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)

In [ ]:
cursor.execute('''SELECT * 
               FROM schedule 
               ORDER BY 取得時間 DESC
               LIMIT 10''')
cols = [col.name for col in cursor.description]
schedule_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(schedule_df)